# Levantamiento Tablas Acciones Comerciales - Querys de CIM desde Teradata

> **PROYECTO : LEVANTAMIENTO CÓDIGO CIM EN EL GESTOR DE CAMPANAS TERADATA** <br> 
**Extracción de tablas utilizadas directamente desde CIM según una lista de comunicaciones específica** <br>
El archivo __query_segmentos_CIM_prod.txt__ tiene la query que extrae las comunicaciones, segmentaciones y su ultima query ejecutada <br>
Versión:  1.0  <br>
Fecha: 01-05-2020  <br>
Descripción: Versión Inicial  <br>
Desarrollador: Axity | Adriana Jiménez 

## Librerías

In [56]:
import pandas as pd
import re
import datetime

## Librerias de parseo de queries
import sqlparse
from sqlparse.sql import TokenList
from sqlparse.tokens import Name, Whitespace, Wildcard, Number, Punctuation, Text, Operator
from sqlparse.tokens import DML, DDL, Keyword
import sql_metadata as sqllib

## Libreria que se integra con Teradata
import giraffez

## Funciones: Insertar en BD Teradata

In [57]:
## Configuracion de conexion
td_config = {
    "username": "exajibl",
    "password": "acjb0610",
    "host": "dataware.bci.cl"}

In [58]:
def create_and_drop_tables_teradata():
    '''
    Función que crea las tablas en Teradata
    '''
    
    print("ELIMINANDO Y CREANDO TABLAS..\n")
    
    drop_sql_bteq    = "DROP TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_CIM"

    create_sql_bteq = """CREATE MULTISET TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_CIM
        (
          NOMBRE_COMUNICACION VARCHAR(100) ,
          NOMBRE_SEGMENTO VARCHAR(50),
          ESQUEMA_INPUT VARCHAR(50) ,
          TABLA_INPUT VARCHAR(50)      
          ) ;"""


    with giraffez.Cmd(**td_config) as cmd:
        if cmd.exists("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_CIM"):
            cmd.execute(drop_sql_bteq)

        cmd.execute(create_sql_bteq)

In [59]:
def insert_files_csv_in_teradata():
    '''
    Función que inserta los dataframes a Teradata
    '''
    
    print("INSERTANDO ARCHIVOS A LAS TABLAS..\n")
    
    df_cim = pd.read_csv("LAC_Levantamiento_Tablas_CIM.csv",  sep='|' )  

    if (df_cim.empty == False): 
        with giraffez.BulkLoad("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_CIM", **td_config) as load:
            load.cleanup()
            load.columns = df_cim.columns.tolist()
            for row in df_cim.values.tolist(): 
                load.put([row[0], row[1], row[2], row[3] ])    

## Funciones modificadas de librería sql_metadata

In [60]:
def unique(_list):
    """
    Hace que una lista tenga registro unicos y mantengan el orden
    """
    
    ret = []

    for item in _list:
        if item not in ret:
            ret.append(item)

    return ret

In [61]:
def get_query_tables(query):
    """
    Función que retorna una lista de tablas INPUT de una query
    """
    tables              = []
    tables_2            = []
    table_list          = []
    table_list_total    = []
    last_keyword  = None
    last_token    = None
    dml_ddl_name  = None
    not_get_from  = False
    from_clausule = False


    df_internal_table   = emptyDataframeTablesQuery()
    
    #     
    dates_list = ['SECOND', 'MINUTE', 'HOUR', 'DAY', 'YEAR','MONTH','BOTH', 'TRAILING']

    
    stop_from_list = ['AS','CASE','WHEN','ON','AND','END','WHERE','GROUP','OVER','PARTITION','SET']
    
    functions_ignored = ['COUNT', 'MIN', 'MAX', 'SUM', 'FROM_UNIXTIME', 'DEC', 
    'CAST', 'CONVERT', 'ZEROIFNULL','SUBSTR','SUBSTRING','ROW_NUMBER','QUALIFY', 'ADD_MONTHS',
    'COALESCE', 'CHAR', 'INTEGER', 'TRIM', 'OVER', 'FORMAT', 'DATE_FORMAT',
    'CHAR_LENGTH']

    # Lista de funciones reservadas de SQL ANSI para las uniones de tablas
    table_syntax_joins = ['FROM','JOIN', 'INNER JOIN', 'LEFT JOIN', 'LEFT OUTER JOIN', 
    'RIGHT JOIN', 'RIGHT OUTER JOIN']

    table_syntax_keywords = [
        # SELECT queries
        'FROM', 'WHERE', 'JOIN', 'INNER JOIN', 'LEFT JOIN', 'LEFT OUTER JOIN', 
        'RIGHT JOIN', 'RIGHT OUTER JOIN', 'ON',
        
        # INSERT queries
        'INTO', 'VALUES',
        # UPDATE queries
        'UPDATE', 'SET',
        # Hive queries
        'TABLE',  # INSERT TABLE
    ]
    
    archivo_caido = 'SQLAExport1383.txt'
    
    for token in sqllib.get_query_tokens(query):
        
        token_value_clean = " ".join(token.value.upper().split())
        
        ## Homologar en caso que se consiga un 'SEL' en vez de 'SELECT'
        if ((token.ttype is Name and token.value.upper() == 'SEL')):
            token.ttype = DML
            token.value = 'SELECT'
            last_keyword = 'SELECT'
            
        if (token.ttype is DML or token.ttype is DDL):
            dml_ddl_name = token.value.upper()  
            from_clausule = False      
        
        if token.ttype is Punctuation and token.value == ',' \
            and from_clausule == True:
            last_keyword = 'FROM'
        
        if token.is_keyword and from_clausule == True and \
           token.value.upper() in stop_from_list:            
            from_clausule = False
            last_keyword = None
                        
        if token.is_keyword and token_value_clean in table_syntax_keywords:
            # keep the name of the last keyword, the next one can be a table name
            last_keyword = " ".join(token.value.upper().split())
            
            ## Para los casos de algunos campos en el SELECT que llaman a FROM
            if token.value.upper() == 'FROM' and last_token in dates_list:
                not_get_from = True
                
            elif(token.value.upper() == 'FROM' and not_get_from == True):
                not_get_from = False
                
            ## Para los casos de que las tablas estén separadas por ,    
            elif(token.value.upper() == 'FROM' and last_token in ["'"+'0'+"'"]):
                from_clausule = False
                not_get_from  = True

            elif(token.value.upper() == 'FROM' and last_token not in["'"+'0'+"'"]):
                from_clausule = True

                
            elif(token.value.upper() in ['WHERE','GROUP']):
                from_clausule = False
            
        elif str(token) == '(' or str(token) == ')':
            #print(" ENTRO 2 " )
            # reset the last_keyword for INSERT `foo` VALUES(id, bar) ...
            last_keyword = None
        elif token.is_keyword and str(token) in ['FORCE', 'ORDER']:
            #print(" ENTRO 3 " )
            # reset the last_keyword for "SELECT x FORCE INDEX" queries and "SELECT x ORDER BY"
            last_keyword = None
        elif token.is_keyword and str(token) == 'SELECT' and last_keyword in ['INTO', 'TABLE']:
            # reset the last_keyword for "INSERT INTO SELECT" and "INSERT TABLE SELECT" queries
            last_keyword = None
            #print(" ENTRO 4 " )
        elif token.ttype is Name or token.is_keyword:
            # print([last_keyword, last_token, token.value])
            # analyze the name tokens, column names and where condition values

            if token.ttype is Name and token.value in functions_ignored \
               and from_clausule == True :
                not_get_from = True


            if last_keyword in ['FROM', 'JOIN', 'INNER JOIN', 'LEFT JOIN', 'RIGHT JOIN',
                                'LEFT OUTER JOIN','RIGHT OUTER JOIN', 'UPDATE' ] \
                    and last_token not in ['AS', 'CREATE'] \
                    and token.value not in ['AS', 'SELECT'] \
                    and not_get_from == False:

                if last_token == '.':
                    
                    # we have database.table notation example
                    # append table name to the last entry of tables
                    # as it is a database name in fact
                    database_name = tables[-1]
                    table = token.value
                    tables[-1] = '{}.{}'.format(database_name, token)
                    last_keyword = None
                    
                    table_list = [database_name, table]
                    if (dml_ddl_name not in ['DROP']):
                        table_list_total.append(table_list)


                elif last_token not in [',', last_keyword] and \
                    last_keyword not in table_syntax_joins:
                    # it's not a list of tables, e.g. SELECT * FROM foo, bar
                    # hence, it can be the case of alias without AS, e.g. SELECT * FROM foo bar
                    pass
                else: #esquema 
                        
                    table_name = str(token.value.strip('`'))
                    tables.append(table_name)
                
                             
        last_token = token.value.upper()

    if (len(table_list_total) == 0):
        result = [ noConditionInput, noConditionInput ]
        table_list_total.append(result)
    
    return unique(table_list_total)   

## Funciones de procesamiento de query

In [62]:
def remove_comments(text):
    """ remove c-style comments.
        text: blob of text with comments (can include newlines)
        returns: text with comments removed
    """
    pattern = r"""
                            ##  --------- COMMENT ---------
           /\*              ##  Start of /* ... */ comment
           [^*]*\*+         ##  Non-* followed by 1-or-more *'s
           (                ##
             [^/*][^*]*\*+  ##
           )*               ##  0-or-more things which don't start with /
                            ##    but do end with '*'
           /                ##  End of /* ... */ comment
         |                  ##  -OR-  various things which aren't comments:
           (                ## 
                            ##  ------ " ... " STRING ------
             "              ##  Start of " ... " string
             (              ##
               \\.          ##  Escaped char
             |              ##  -OR-
               [^"\\]       ##  Non "\ characters
             )*             ##
             "              ##  End of " ... " string
           |                ##  -OR-
                            ##
                            ##  ------ ' ... ' STRING ------
             '              ##  Start of ' ... ' string
             (              ##
               \\.          ##  Escaped char
             |              ##  -OR-
               [^'\\]       ##  Non '\ characters
             )*             ##
             '              ##  End of ' ... ' string
           |                ##  -OR-
                            ##
                            ##  ------ ANYTHING ELSE -------
             .              ##  Anything other char
             [^/"'\\]*      ##  Chars which doesn't start a comment, string
           )                ##    or escape
    """
    regex = re.compile(pattern, re.VERBOSE|re.MULTILINE|re.DOTALL)
    noncomments = [m.group(2) for m in regex.finditer(text) if m.group(2)]

    return "".join(noncomments)

In [63]:
def deleteInconsistencyQuery(queryString):
    '''
    Función que elimina caracteres varios de una query
    '''    
    characters = '[%#${}]'
    finalQuery = re.sub(characters, '', str(queryString))
    return finalQuery

## Funciones: Insertar en Dataframes

In [64]:
def insert_dataframe_in_csv(df):
    '''
    Función que crea un archivo de levantamiento desde un dataframe
    '''
    
    with open('LAC_Levantamiento_Tablas_CIM.csv', 'a') as f:
        df.to_csv(f, sep='|', index=None, header=f.tell()==0)

In [65]:
def emptyDataframeTablesQuery():
    '''
    Función que retorna un objeto dataframe vacío para obtener las tablas de una query
    
    Formato ['NOMBRE_COMUNICACION','NOMBRE_SEGMENTO','ESQUEMA_INPUT','TABLA_INPUT']
    '''
    
    columnsDF = ['NOMBRE_COMUNICACION','NOMBRE_SEGMENTO','ESQUEMA_INPUT','TABLA_INPUT']

    df = pd.DataFrame(columns = columnsDF) 
    return df

In [66]:
def insertDataframeTablesQuery(communication, segment, input_tables_list):
    '''
    Función que inserta en un dataframe los objetos de una query
    '''

    new_df_tables = emptyDataframeTablesQuery()
    
    if (input_tables_list):
        for item in input_tables_list:
            
            new_df_tables = new_df_tables.append(
                [ { 'NOMBRE_COMUNICACION' : communication, 
                    'NOMBRE_SEGMENTO'   : segment,
                    'ESQUEMA_INPUT'  : item[0],
                    'TABLA_INPUT'    : item[1] }], 
                    ignore_index=True, sort=False) 
   
    return new_df_tables

## Query desde TERADATA

In [67]:
def get_querys_cim_teradata():
    '''
    Función que busca en Teradata las querys a procesar según comunicación y segmentación
    '''

    filename        = 'query_segmentos_CIM_prod.txt'
    archivo         = open(filename, encoding="utf8",errors='ignore')
    query_segments  = archivo.read()
    
    with giraffez.BulkExport(query_segments, **td_config) as export:
        dataset = export.to_list()
        
    headers = ['NOMBRE_COMUNICACION','SEGMENTO', 'QUERY_CIM', 'ULTIMA_FECHA']
    #df = pd.DataFrame(headers, dataset)

    df = pd.DataFrame(dataset, columns = headers)
    return df

In [ ]:
if __name__ == "__main__":
    
    now = datetime.datetime.now()
    print (now)
    
    # Obtiene en un dataframe la información
    df = get_querys_cim_teradata()
    query_cim_list = df.values.tolist()
   
    df_internal_table_file = emptyDataframeTablesQuery()


    # Recorre las comunicaciones -> segmentaciones -> query 
    for query in query_cim_list:

        communication = query[0]
        segment       = query[1]
        queryString   = query[2]

        queryString = queryString.replace('"','')

        queryString = deleteInconsistencyQuery(queryString)

        queryString = re.sub('\-\-.*?\n|\/\*.*?\*\/', ' ', queryString)  

        queryString = remove_comments(queryString)

        # Retorna todas las tablas input
        input_tables_list = get_query_tables(queryString)

        # Retorna todas las tablas input en forma de dataframe con su comunicacion y segmentacion
        df_internal_table = insertDataframeTablesQuery(communication, segment, input_tables_list)

        # Acumula los df en uno final
        if (df_internal_table.empty == False):
            df_internal_table_file = df_internal_table_file.append(df_internal_table)

    
    # Inserta todo el analisis de extracción de tablas a un archivo .CSV
    insert_dataframe_in_csv(df_internal_table_file)  
    
    '''
    Sección donde crea e inserta en la tabla temporal de Teradata la información obtenida
    '''
        
    # Elimina y crea las tablas en Teradata
    create_and_drop_tables_teradata()
    
    # Inserta el contenido de los archivos a las tablas en Teradata
    insert_files_csv_in_teradata()
    
    now = datetime.datetime.now()
    print (now)